In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/NLP_Project/fulltrain.csv'

column_names = ['label', 'text']

df = pd.read_csv(file_path, names=column_names)

df.head()


,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


Swear Words Analysis & Insights

In [ ]:
swear_words_df_2 = pd.read_csv('/content/drive/MyDrive/NLP_Project/profanity_en.csv')

In [ ]:
swear_words_df_2.head()

,text,canonical_form_1,canonical_form_2,canonical_form_3,category_1,category_2,category_3,severity_rating,severity_description
0,69,69,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
1,@55,ass,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
2,@ssfcker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.8,Severe
3,@ssfucker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.8,Severe
4,@ssfvcker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.4,Strong


In [ ]:
swear_words_df_2['category_1'].value_counts()

sexual anatomy / sexual acts    980
racial / ethnic slurs           192
sexual orientation / gender     182
bodily fluids / excrement       146
other / general insult           52
religious offense                19
mental disability                15
animal references                 5
political                         3
physical attributes               3
physical disability               1
Name: category_1, dtype: int64

In [ ]:
swear_words_df_2['category_2'].value_counts()

other / general insult          206
sexual orientation / gender     123
bodily fluids / excrement        61
animal references                33
mental disability                27
racial / ethnic slurs            12
sexual anatomy / sexual acts     12
physical attributes              10
religious offense                 2
Name: category_2, dtype: int64

In [ ]:
swear_words_df_2 = swear_words_df_2['canonical_form_1'].unique().tolist()

In [ ]:
print(len(swear_words_df_2))

244


In [ ]:
swear_words_dict = dict(zip(swear_words_df_2['canonical_form_1'], swear_words_df_2['severity_rating']))


In [ ]:
import re


# Function to calculate total severity rating for a text
def get_total_severity(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive matching
    total_severity = 0
    for word in swear_words_dict:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            total_severity += swear_words_dict[word]
    return total_severity

# Initialize dictionaries to store counts, percentages, and total severity ratings by class
counts_by_class = {}
percentages_by_class = {}
total_severity_by_class = {}

# Calculate counts, percentages, and total severity ratings by class
total_rows = len(df)
for label in df['label'].unique():
    print("processing class: ", label)
    class_rows = df[df['label'] == label]
    class_rows_with_swear_words = 0
    total_severity = 0
    tot = len(class_rows)
    counter = 0
    for text in class_rows['text']:
        if counter % 1000 == 0:
            print("row: ", counter)
        counter += 1
        total_severity += get_total_severity(text)
        if get_total_severity(text) > 0:
            class_rows_with_swear_words += 1
    counts_by_class[label] = class_rows_with_swear_words
    percentages_by_class[label] = (class_rows_with_swear_words / len(class_rows)) * 100
    total_severity_by_class[label] = total_severity / len(class_rows)

# Print class-wise counts, percentages, and total severity ratings
for label, count in counts_by_class.items():
    percentage = percentages_by_class[label]
    total_severity = total_severity_by_class[label]
    print(f"Class '{label}':")
    print(f"Number of rows containing swear words: {count}")
    print(f"Percentage of rows containing swear words: {percentage:.2f}%")
    print(f"Average severity rating of swear words in the class: {total_severity:.2f}")
    print()

# Total counts and percentages
total_rows_with_swear_words = sum(counts_by_class.values())
total_percentage = (total_rows_with_swear_words / total_rows) * 100
print("Total:")
print(f"Total number of rows containing swear words: {total_rows_with_swear_words}")
print(f"Total percentage of rows containing swear words: {total_percentage:.2f}%")

processing class:  1
row:  0
row:  1000
row:  2000
row:  3000
row:  4000
row:  5000
row:  6000
row:  7000
row:  8000
row:  9000
row:  10000
row:  11000
row:  12000
row:  13000
row:  14000
processing class:  2
row:  0
row:  1000
row:  2000
row:  3000
row:  4000
row:  5000
row:  6000
processing class:  3
row:  0
row:  1000
row:  2000
row:  3000
row:  4000
row:  5000
row:  6000
row:  7000
row:  8000
row:  9000
row:  10000
row:  11000
row:  12000
row:  13000
row:  14000
row:  15000
row:  16000
row:  17000
processing class:  4
row:  0
row:  1000
row:  2000
row:  3000
row:  4000
row:  5000
row:  6000
row:  7000
row:  8000
row:  9000
Class '1':
Number of rows containing swear words: 3509
Percentage of rows containing swear words: 24.98%
Average severity rating of swear words in the class: 0.72

Class '2':
Number of rows containing swear words: 657
Percentage of rows containing swear words: 9.46%
Average severity rating of swear words in the class: 0.21

Class '3':
Number of rows containing sw

In [ ]:
# checked before pre-processing
# Function to check if a text contains '*'
def contains_star(text):
    return '*' in text

# Calculate the number of rows containing '*'
num_rows_with_star = sum(df['text'].apply(contains_star))

print(f"Number of rows containing '*': {num_rows_with_star}")


Number of rows containing '*': 1070


Preprocess

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = df['text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
df['processed_text'] = preprocessed_corpus


LDA

printing top 20 words

(after analysis we are sticking to 5 topics, either the one from 20 words printing or 50 words printing, but 5 topics ony. 10 and 20 cause overlap and not much separation between classes)

In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=5, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 20):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.011*"percent" + 0.007*"year" + 0.005*"last" + 0.005*"two" + 0.005*"first" + 0.005*"taiwans" + 0.004*"new" + 0.004*"million" + 0.004*"city" + 0.004*"market" + 0.004*"team" + 0.003*"three" + 0.003*"taipei" + 0.003*"also" + 0.003*"tuesday" + 0.003*"chinese" + 0.003*"wednesday" + 0.003*"billion" + 0.003*"friday" + 0.003*"points"

Topic: 1 
Words: 0.006*"police" + 0.005*"one" + 0.004*"court" + 0.004*"people" + 0.004*"media" + 0.004*"school" + 0.004*"news" + 0.003*"also" + 0.003*"would" + 0.003*"public" + 0.003*"case" + 0.003*"students" + 0.003*"new" + 0.003*"like" + 0.003*"show" + 0.002*"information" + 0.002*"two" + 0.002*"time" + 0.002*"law" + 0.002*"first"

Topic: 2 
Words: 0.007*"people" + 0.007*"like" + 0.007*"one" + 0.006*"get" + 0.005*"dont" + 0.005*"would" + 0.005*"time" + 0.004*"know" + 0.004*"make" + 0.004*"even" + 0.004*"going" + 0.004*"good" + 0.004*"years" + 0.003*"way" + 0.003*"see" + 0.003*"much" + 0.003*"want" + 0.003*"go" + 0.003*"could" + 0.003*"think"

T

Based on the top words associated with each topic, here are the topics we could assign for our understanding(tentative):

Topic 0: Sports or Entertainment

Words like "team," "game," "season," "players," and "play" suggest a topic related to sports or entertainment. Terms like "Taipei" and "Taiwan" might indicate a focus on Taiwanese sports teams or events.

Topic 1: Economics or Politics

Words like "percent," "economic," "government," "China," "million," "billion," and "market" suggest a topic related to economics or politics. References to "Taiwan" and "US" further support this interpretation.

Topic 2: Health or Medicine

Words like "police," "vaccine," "medical," "children," "health," and "death" suggest a topic related to health or medicine. Terms like "found" and "cases" could indicate discussions about medical discoveries or disease outbreaks.

Topic 3: Opinions or Sentiments

Words like "like," "people," "get," "dont," "think," "going," "want," "even," "good," "make," "see," "go," "thats," and "really" suggest a topic related to opinions or sentiments. This topic might encompass discussions about personal
experiences, attitudes, or beliefs.

Topic 4: Government or Politics

Words like "government," "president," "state," "military," "war," "law," "United," "security," "political," "officials," and "American" suggest a topic related to government or politics. References to "US," "Obama," and "United States" further support this interpretation.

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame containing the text data and classes

# Step 1: Assign topics to rows
topic_assignments = []
for idx, row in df.iterrows():
    bow_vector = dictionary.doc2bow(row['processed_text'])
    topic_distribution = lda_model.get_document_topics(bow_vector)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_assignments.append(dominant_topic)

df['topic'] = topic_assignments

# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic          0          1          2          3          4
label                                                       
1      14.323343  18.345554  47.647184  11.589663   8.094255
2       0.677038  40.262172  12.013829  46.110631   0.936330
3       1.768327  19.697818  13.922776  35.847790  28.763290
4      30.035018  10.875438   9.044522  33.696848  16.348174

Percentage of topics per class:
label          1          2          3          4
topic                                            
0      37.418635   0.874093   5.876883  55.830389
1      25.824231  28.008819  35.274076  10.892875
2      61.296822   7.638062  22.785969   8.279146
3      11.148394  21.920153  43.867698  23.063754
4      14.255266   0.814945  64.443330  20.486459


In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=10, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 20):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.011*"government" + 0.008*"us" + 0.007*"president" + 0.007*"would" + 0.006*"state" + 0.006*"people" + 0.006*"states" + 0.006*"obama" + 0.004*"political" + 0.004*"law" + 0.004*"public" + 0.004*"american" + 0.004*"national" + 0.004*"united" + 0.004*"new" + 0.003*"federal" + 0.003*"also" + 0.003*"one" + 0.003*"bill" + 0.003*"country"

Topic: 1 
Words: 0.010*"people" + 0.010*"like" + 0.009*"one" + 0.007*"dont" + 0.006*"get" + 0.006*"know" + 0.006*"would" + 0.005*"time" + 0.004*"think" + 0.004*"school" + 0.004*"see" + 0.004*"even" + 0.004*"want" + 0.004*"way" + 0.004*"good" + 0.004*"life" + 0.004*"im" + 0.004*"really" + 0.004*"make" + 0.003*"much"

Topic: 2 
Words: 0.010*"us" + 0.006*"military" + 0.005*"taiwan" + 0.005*"government" + 0.004*"would" + 0.004*"war" + 0.004*"also" + 0.004*"security" + 0.004*"syria" + 0.004*"china" + 0.003*"one" + 0.003*"people" + 0.003*"minister" + 0.003*"united" + 0.003*"two" + 0.003*"foreign" + 0.003*"officials" + 0.003*"international" + 0.00


Based on the words associated with each topic, here are the topics that I would assign:

Topic 0: Government and Politics
Words like "government," "president," "state," "law," "public," and "national" suggest a topic related to government and politics.

Topic 1: Personal Experiences and Opinions
Words like "people," "like," "dont," "know," "time," "think," and "want" suggest a topic related to personal experiences and opinions.

Topic 2: International Affairs and Military
Words like "us," "military," "taiwan," "china," "war," and "security" suggest a topic related to international affairs and military matters.

Topic 3: Food and Nutrition
Words like "food," "water," "beef," "eat," "organic," and "products" suggest a topic related to food and nutrition.

Topic 4: Sports and Events
Words like "taiwans," "taipei," "team," "games," "year," and "players" suggest a topic related to sports and events.

Topic 5: Economics and Finance
Words like "percent," "economic," "market," "financial," "money," "company," and "bank" suggest a topic related to economics and finance.

Topic 6: Legal Matters and Judiciary
Words like "court," "case," "law," "judge," "attorney," "rights," and "supreme" suggest a topic related to legal matters and the judiciary.

Topic 7: Medical Research and Health
Words like "medical," "research," "health," "drug," "vaccine," "science," and "disease" suggest a topic related to medical research and health.

Topic 8: Environmental Issues and Health
Words like "cancer," "oil," "energy," "climate," "environmental," "vaccines," and "disease" suggest a topic related to environmental issues and health.

Topic 9: General Events and News
Words like "two," "first," "last," "team," "time," "city," "game," "car," and "police" suggest a topic related to general events and news.

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame containing the text data and classes

# Step 1: Assign topics to rows
topic_assignments = []
for idx, row in df.iterrows():
    bow_vector = dictionary.doc2bow(row['processed_text'])
    topic_distribution = lda_model.get_document_topics(bow_vector)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_assignments.append(dominant_topic)

df['topic'] = topic_assignments

# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic          0          1          2         3         4          5  \
label                                                                   
1       9.453976  35.772763   4.449349  4.235780  3.182174   6.691820   
2      41.817920  24.805532  12.791703  0.950735  0.172861   1.224431   
3      18.852826  13.447118  18.461108  3.939564  0.811416  20.201455   
4      14.097049   7.833917  21.980990  2.321161  7.893947  26.163082   

topic          6          7         8          9  
label                                             
1       3.032676   5.317861  1.345483  26.518118  
2      10.357246   1.008355  0.100835   6.770383  
3       3.883604  12.092893  6.838276   1.471740  
4       3.991996   3.771886  2.721361   9.224612  

Percentage of topics per class:
label          1          2          3          4
topic                                            
0      14.740815  32.223332  37.395937  15.639916
1      50.588946  17.336

In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=20, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 20):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.009*"israel" + 0.008*"trump" + 0.007*"attacks" + 0.007*"israeli" + 0.007*"attack" + 0.005*"terrorist" + 0.005*"us" + 0.005*"election" + 0.005*"911" + 0.005*"militants" + 0.004*"border" + 0.004*"would" + 0.004*"taliban" + 0.004*"gaza" + 0.004*"palestinian" + 0.004*"campaign" + 0.004*"also" + 0.004*"news" + 0.004*"one" + 0.004*"people"

Topic: 1 
Words: 0.013*"airport" + 0.011*"flight" + 0.010*"air" + 0.009*"travel" + 0.009*"flights" + 0.008*"passengers" + 0.008*"plane" + 0.007*"airlines" + 0.006*"haiti" + 0.006*"earthquake" + 0.005*"two" + 0.005*"tourists" + 0.005*"baseball" + 0.005*"aircraft" + 0.004*"fly" + 0.004*"rays" + 0.004*"airline" + 0.004*"golf" + 0.004*"planes" + 0.004*"flying"

Topic: 2 
Words: 0.107*"taiwan" + 0.038*"chinese" + 0.036*"china" + 0.014*"chinas" + 0.007*"beijing" + 0.006*"province" + 0.006*"fraud" + 0.005*"hu" + 0.004*"group" + 0.004*"report" + 0.004*"also" + 0.003*"premier" + 0.003*"enterprises" + 0.003*"monday" + 0.003*"two" + 0.003*"influen

Based on the words associated with each topic, here are the topics that I would assign:

Topic 1: Travel and Aviation

Topic 2: Taiwan and China Relations

Topic 3: Health and Medical Research

Topic 4: Holidays and Festivities

Topic 5: Politics and Government

Topic 6: Entertainment and Arts

Topic 7: Information Technology and Data

Topic 8: Military and International Affairs

Topic 9: Government Policies and Economics

Topic 10: Global Economics and Financial System

Topic 11: Law Enforcement and Crime

Topic 12: Sports and Events

Topic 13: Crime and Violence

Topic 14: Miscellaneous Topics

Topic 15: Personal Experiences and Opinions

Topic 16: Environmental Issues and Climate

Topic 17: Food and Agriculture

Topic 18: Education and School Safety

Topic 19: Legal Decisions and Court Cases

In [ ]:

import pandas as pd

# Assuming 'df' is your DataFrame containing the text data and classes

# Step 1: Assign topics to rows
topic_assignments = []
for idx, row in df.iterrows():
    bow_vector = dictionary.doc2bow(row['processed_text'])
    topic_distribution = lda_model.get_document_topics(bow_vector)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_assignments.append(dominant_topic)

df['topic'] = topic_assignments

# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic        0         1         2          3         4          5         6   \
label                                                                           
1      1.067844  1.025130  0.149498   3.310315  0.242045   5.780594  5.125650   
2      9.348891  0.144051  0.000000   0.677038  0.187266  30.048977  3.154710   
3      1.913822  0.257415  0.229435  12.898713  0.184667   3.799664  0.755456   
4      1.620810  1.310655  0.920460   3.661831  0.210105   7.503752  4.492246   

topic        7          8          9          10        11         12  \
label                                                                   
1      2.755037   3.822880   3.602193   3.246245  1.665836  14.643696   
2      0.532988   8.945549   3.961394   0.993950  5.257851   2.765774   
3      8.931170  21.611640   8.282037  11.275881  3.626189   0.822608   
4      3.301651  19.579790  22.111056   5.022511  3.471736   9.114557   

topic        13        14 

top 50 words

In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=5, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 50):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.006*"two" + 0.005*"first" + 0.005*"city" + 0.005*"last" + 0.005*"team" + 0.004*"taiwans" + 0.004*"taipei" + 0.004*"year" + 0.004*"three" + 0.004*"new" + 0.004*"one" + 0.003*"years" + 0.003*"game" + 0.003*"also" + 0.003*"wednesday" + 0.003*"four" + 0.003*"season" + 0.003*"friday" + 0.003*"second" + 0.003*"tuesday" + 0.003*"time" + 0.003*"games" + 0.003*"thursday" + 0.002*"sunday" + 0.002*"players" + 0.002*"world" + 0.002*"monday" + 0.002*"million" + 0.002*"saturday" + 0.002*"would" + 0.002*"play" + 0.002*"told" + 0.002*"week" + 0.002*"south" + 0.002*"day" + 0.002*"five" + 0.002*"made" + 0.002*"teams" + 0.002*"top" + 0.002*"back" + 0.002*"former" + 0.002*"next" + 0.002*"league" + 0.002*"county" + 0.002*"home" + 0.002*"national" + 0.002*"win" + 0.002*"10" + 0.002*"los" + 0.002*"angeles"

Topic: 1 
Words: 0.013*"percent" + 0.005*"year" + 0.004*"us" + 0.004*"billion" + 0.004*"market" + 0.004*"one" + 0.004*"food" + 0.004*"health" + 0.003*"would" + 0.003*"million" + 0.003*"

Based on the words associated with each topic, here are the topics that I would assign:

Topic 0: Sports and Events

Topic 1: Economics and Financial Markets

Topic 2: Personal Experiences and Opinions

Topic 3: Politics and Government Policies

Topic 4: International Relations and Foreign Affairs

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame containing the text data and classes

# Step 1: Assign topics to rows
topic_assignments = []
for idx, row in df.iterrows():
    bow_vector = dictionary.doc2bow(row['processed_text'])
    topic_distribution = lda_model.get_document_topics(bow_vector)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_assignments.append(dominant_topic)

df['topic'] = topic_assignments

# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic          0          1          2          3          4
label                                                       
1      14.323343  18.345554  47.647184  11.589663   8.094255
2       0.677038  40.262172  12.013829  46.110631   0.936330
3       1.768327  19.697818  13.922776  35.847790  28.763290
4      30.035018  10.875438   9.044522  33.696848  16.348174

Percentage of topics per class:
label          1          2          3          4
topic                                            
0      37.418635   0.874093   5.876883  55.830389
1      25.824231  28.008819  35.274076  10.892875
2      61.296822   7.638062  22.785969   8.279146
3      11.148394  21.920153  43.867698  23.063754
4      14.255266   0.814945  64.443330  20.486459


In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=10, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 50):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.011*"president" + 0.010*"obama" + 0.007*"house" + 0.006*"party" + 0.006*"would" + 0.005*"news" + 0.005*"white" + 0.005*"people" + 0.005*"one" + 0.005*"clinton" + 0.005*"election" + 0.004*"bill" + 0.004*"campaign" + 0.004*"former" + 0.004*"like" + 0.004*"think" + 0.004*"democratic" + 0.004*"media" + 0.003*"republican" + 0.003*"time" + 0.003*"political" + 0.003*"nsa" + 0.003*"vote" + 0.003*"presidential" + 0.003*"hillary" + 0.003*"know" + 0.003*"senate" + 0.003*"office" + 0.003*"first" + 0.003*"democrats" + 0.003*"dont" + 0.003*"new" + 0.003*"public" + 0.003*"also" + 0.003*"us" + 0.003*"say" + 0.003*"candidate" + 0.002*"get" + 0.002*"could" + 0.002*"committee" + 0.002*"washington" + 0.002*"republicans" + 0.002*"obamas" + 0.002*"made" + 0.002*"going" + 0.002*"show" + 0.002*"post" + 0.002*"hes" + 0.002*"want" + 0.002*"voters"

Topic: 1 
Words: 0.007*"get" + 0.006*"one" + 0.005*"like" + 0.005*"time" + 0.005*"food" + 0.005*"home" + 0.004*"back" + 0.004*"dont" + 0.004*"make

Based on the words associated with each topic, here are the topics that I would assign:

Topic 0: Political Figures and Elections

Topic 1: Daily Life and Activities

Topic 2: Urban and Environmental Issues

Topic 3: Economics and Financial Markets

Topic 4: Technology and Information

Topic 5: International Relations and Conflicts

Topic 6: Health and Medical Topics

Topic 7: Legal and Law Enforcement Issues

Topic 8: Financial Markets and Trading

Topic 9: Sports and Entertainment

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame containing the text data and classes

# Step 1: Assign topics to rows
topic_assignments = []
for idx, row in df.iterrows():
    bow_vector = dictionary.doc2bow(row['processed_text'])
    topic_distribution = lda_model.get_document_topics(bow_vector)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_assignments.append(dominant_topic)

df['topic'] = topic_assignments

# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic          0          1          2         3         4          5  \
label                                                                   
1       9.453976  35.772763   4.449349  4.235780  3.182174   6.691820   
2      41.817920  24.805532  12.791703  0.950735  0.172861   1.224431   
3      18.852826  13.447118  18.461108  3.939564  0.811416  20.201455   
4      14.097049   7.833917  21.980990  2.321161  7.893947  26.163082   

topic          6          7         8          9  
label                                             
1       3.032676   5.317861  1.345483  26.518118  
2      10.357246   1.008355  0.100835   6.770383  
3       3.883604  12.092893  6.838276   1.471740  
4       3.991996   3.771886  2.721361   9.224612  

Percentage of topics per class:
label          1          2          3          4
topic                                            
0      14.740815  32.223332  37.395937  15.639916
1      50.588946  17.336

In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=20, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 50):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.024*"health" + 0.013*"medical" + 0.009*"cancer" + 0.007*"disease" + 0.006*"patients" + 0.006*"study" + 0.006*"research" + 0.006*"flu" + 0.005*"people" + 0.005*"cases" + 0.005*"dr" + 0.005*"doctors" + 0.005*"care" + 0.005*"found" + 0.004*"hospital" + 0.004*"treatment" + 0.004*"virus" + 0.004*"one" + 0.004*"may" + 0.004*"healthcare" + 0.004*"death" + 0.004*"years" + 0.004*"published" + 0.003*"1" + 0.003*"risk" + 0.003*"many" + 0.003*"medicine" + 0.003*"also" + 0.003*"studies" + 0.003*"effects" + 0.003*"cause" + 0.003*"body" + 0.003*"us" + 0.003*"diseases" + 0.003*"deaths" + 0.003*"blood" + 0.003*"percent" + 0.003*"dna" + 0.003*"insurance" + 0.003*"new" + 0.003*"human" + 0.003*"two" + 0.003*"report" + 0.002*"drug" + 0.002*"2009" + 0.002*"cells" + 0.002*"women" + 0.002*"since" + 0.002*"doctor" + 0.002*"information"

Topic: 1 
Words: 0.013*"information" + 0.013*"news" + 0.010*"new" + 0.009*"internet" + 0.007*"data" + 0.006*"media" + 0.006*"online" + 0.006*"show" + 0.006*"

Based on the words associated with each topic, here are the topics that I would assign:

Topic 1: Travel and Aviation

Topic 2: Taiwan and China Relations

Topic 3: Health and Medical Research

Topic 4: Holidays and Festivities

Topic 5: Politics and Government

Topic 6: Entertainment and Arts

Topic 7: Information Technology and Data

Topic 8: Military and International Affairs

Topic 9: Government Policies and Economics

Topic 10: Global Economics and Financial System

Topic 11: Law Enforcement and Crime

Topic 12: Sports and Events

Topic 13: Crime and Violence

Topic 14: Miscellaneous Topics

Topic 15: Personal Experiences and Opinions

Topic 16: Environmental Issues and Climate

Topic 17: Food and Agriculture

Topic 18: Education and School Safety

Topic 19: Legal Decisions and Court Cases

In [ ]:

import pandas as pd

# Assuming 'df' is your DataFrame containing the text data and classes

# Step 1: Assign topics to rows
topic_assignments = []
for idx, row in df.iterrows():
    bow_vector = dictionary.doc2bow(row['processed_text'])
    topic_distribution = lda_model.get_document_topics(bow_vector)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_assignments.append(dominant_topic)

df['topic'] = topic_assignments

# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic        0         1         2          3         4          5         6   \
label                                                                           
1      1.067844  1.025130  0.149498   3.310315  0.242045   5.780594  5.125650   
2      9.348891  0.144051  0.000000   0.677038  0.187266  30.048977  3.154710   
3      1.913822  0.257415  0.229435  12.898713  0.184667   3.799664  0.755456   
4      1.620810  1.310655  0.920460   3.661831  0.210105   7.503752  4.492246   

topic        7          8          9          10        11         12  \
label                                                                   
1      2.755037   3.822880   3.602193   3.246245  1.665836  14.643696   
2      0.532988   8.945549   3.961394   0.993950  5.257851   2.765774   
3      8.931170  21.611640   8.282037  11.275881  3.626189   0.822608   
4      3.301651  19.579790  22.111056   5.022511  3.471736   9.114557   

topic        13        14 

##Generate new columns - 'has_swear', 'severity', 'topic'

performing swear words related analysis before punctuation,stopwords removal

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/NLP_Project/fulltrain.csv'

column_names = ['label', 'text']

df = pd.read_csv(file_path, names=column_names)

df.head()


,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [ ]:
swear_words_df_2 = pd.read_csv('/content/drive/MyDrive/NLP_Project/profanity_en.csv')

In [ ]:
swear_words_dict = dict(zip(swear_words_df_2['canonical_form_1'], swear_words_df_2['severity_rating']))


Combining augmented dataset from each team member

In [ ]:
df_aug_1 = pd.read_csv('/content/drive/MyDrive/NLP_Project/1_train_aug.csv')
df_aug_2 = pd.read_csv('/content/drive/MyDrive/NLP_Project/2_train_aug.csv')
df_aug_3 = pd.read_csv('/content/drive/MyDrive/NLP_Project/3_train_aug.csv')
df_aug_4 = pd.read_csv('/content/drive/MyDrive/NLP_Project/4_train_aug.csv')
df_aug_5 = pd.read_csv('/content/drive/MyDrive/NLP_Project/5_train_aug.csv')
df_aug_6 = pd.read_csv('/content/drive/MyDrive/NLP_Project/6_train_aug.csv')

In [ ]:
df_aug_1.head()

,Label,Text
0,2,Ralph Peters Is Suspended By Fox News After Ve...
1,2,"Michelle Obama Picks A Fight With Donald, He F..."
2,2,Analyst Observes Something VERY Off About Trum...
3,2,Marine Completely DESTROYS Michelle Obama To H...
4,2,Youll Never Believe The Excuse This Mother Gav...


In [ ]:
df_aug_2.head()

,Label,Text
0,2,Montel Williams Voices His Opinion On What Sho...
1,2,"Marines Welcome George W. Bush, But Are Silent..."
2,2,BREAKING: Hillary Scrambling After Fox News Bl...
3,2,Muslims Fight To Ban Christianity From The Mil...
4,2,Hillary Clinton Caught Doing THIS With The Sau...


In [ ]:
df_aug_3.head()

,Label,Text
0,2,See Who Donald Trump Just Announced As The Per...
1,2,"Coward Traitor Loretta Lynch Pleads The 5th, I..."
2,2,Obama Sends Dangerous Message To Ferguson Prot...
3,2,BREAKING: How Hillary Clinton Could Still Win ...
4,2,This Guy Started Secretly Filming Trump And Ca...


In [ ]:
df_aug_4.head()

,Label,Text
0,2,Muslims Everywhere Are Furious After DHL Makes...
1,2,Hillary Literally Caught On Live TV Handing Qu...
2,2,BREAKING NEWS: State Votes To Fully Nullify Ob...
3,2,Federal Judge Orders Obama To Court Over Uncon...
4,2,Al Sharpton Assaults Fox News Reporter [VIDEO]...


In [ ]:
df_aug_5.head()

,Label,Text
0,2,Top Democrat Does Something Sickening During T...
1,2,Donald Trump Reveals One Thing About Obama Tha...
2,2,ISIS Releases Statement: We Will Attack Texas ...
3,2,Marine Steps In When Thug Attacks Cop [VIDEO] ...
4,2,BREAKING: Hillary Clinton Faces 3 Long Years I...


In [ ]:
df_aug_6.head()

,Label,Text
0,2,Dolly Parton Just Insulted Christians Across T...
1,2,Rancher Lavoy Finicum Shot And Killed By Feds ...
2,2,Burqa Wearing Muslim Woman Berating Cashier Ge...
3,2,Zoe Saldana Risks Her Entire Career To Defend ...
4,2,"Mother Of Benghazi Victim Breaks Her Silence, ..."


In [ ]:
filtered_orig_df = df[(df['label'] == 1) | (df['label'] == 3)]

# If you want to reset index
filtered_orig_df.reset_index(drop=True, inplace=True)

# Show the resulting DataFrame
print(filtered_orig_df)

In [ ]:
concatenated_aug_df = pd.concat([df_aug_1, df_aug_2, df_aug_3, df_aug_4, df_aug_5, df_aug_6], ignore_index=True)


In [ ]:
concatenated_aug_df = concatenated_aug_df.rename(columns={'Label': 'label', 'Text': 'text'})


In [ ]:
concatenated_aug_df.head()

,label,text
0,2,Ralph Peters Is Suspended By Fox News After Ve...
1,2,"Michelle Obama Picks A Fight With Donald, He F..."
2,2,Analyst Observes Something VERY Off About Trum...
3,2,Marine Completely DESTROYS Michelle Obama To H...
4,2,Youll Never Believe The Excuse This Mother Gav...


In [ ]:
merged_final_df = pd.concat([filtered_orig_df, concatenated_aug_df])

In [ ]:
merged_final_df['label'].value_counts()

label
3    17870
1    14047
2    14046
4    14043
Name: count, dtype: int64

In [ ]:
merged_final_df.describe()

,label
count,60006.000000
mean,2.531763
std,1.088692
min,1.000000
25%,2.000000
50%,3.000000
75%,3.000000
max,4.000000


In [ ]:
merged_final_df = merged_final_df.dropna(subset=['text'])


In [ ]:
merged_final_df['label'].value_counts()

label
3    17870
1    14047
2    14046
4    14037
Name: count, dtype: int64

In [ ]:
text_len = merged_final_df['text'].str.split().apply(len)

# Find the minimum length
min_length = text_len.min()

# Find the maximum length
max_length = text_len.max()

# Find the average length
avg_length = text_len.mean()

# Print the results
print("Minimum text length:", min_length)
print("Maximum text length:", max_length)
print("Average text length:", avg_length)

Minimum text length: 1
Maximum text length: 117495
Average text length: 467.15976666666666


In [ ]:
merged_final_df['word_count'] = merged_final_df['text'].apply(lambda x: len(str(x).split()))

# Filter out rows where word count is less than 5
merged_final_df = merged_final_df[merged_final_df['word_count'] >= 5]

# Drop the 'word_count' column if it's no longer needed
merged_final_df.drop(columns=['word_count'], inplace=True)

# Print the resulting DataFrame
print(merged_final_df)

       label                                               text
0          1  A little less than a decade ago, hockey fans w...
1          1  The writers of the HBO series The Sopranos too...
2          1  Despite claims from the TV news outlet to offe...
3          1  After receiving 'subpar' service and experienc...
4          1  After watching his beloved Seattle Mariners pr...
...      ...                                                ...
28084      4   A delegation of approximately 500 to 1,000 in...
28085      4   Taiwan's Lu Yen-Hsun and his partner Janko Ti...
28086      4   Democratic candidate Martha Coakley and Repub...
28087      4   Holding by-elections frequently due to electi...
28088      4   Renowned comedian, known for his zany televis...

[59795 rows x 2 columns]


<ipython-input-39-141fce3e0a79>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final_df.drop(columns=['word_count'], inplace=True)


In [ ]:
merged_final_df['label'].value_counts()

label
3    17730
1    14036
2    14022
4    14007
Name: count, dtype: int64

###adding features to the new merged df

In [ ]:
merged_final_df.head()

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [ ]:
import re
from tqdm import tqdm

# Function to calculate total severity rating for a text
def get_total_severity(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive matching
    total_severity = 0
    for word in swear_words_dict:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            total_severity += swear_words_dict[word]
    return total_severity

# Function to check if a text contains any swear words
def contains_swear_words(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive matching
    for word in swear_words_dict:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            return True
    return False

# Add 'has_swear_word' and 'severity' columns to the DataFrame
tqdm.pandas(desc="Applying contains_swear_words")
merged_final_df['has_swear_word'] = merged_final_df['text'].progress_apply(contains_swear_words)
tqdm.pandas(desc="Applying get_total_severity")
merged_final_df['severity'] = merged_final_df['text'].progress_apply(get_total_severity)

# Print the modified DataFrame
merged_final_df.head()

Applying get_total_severity: 100%|██████████| 59795/59795 [19:50<00:00, 50.25it/s] 


,label,text,has_swear_word,severity
0,1,"A little less than a decade ago, hockey fans w...",False,0.0
1,1,The writers of the HBO series The Sopranos too...,False,0.0
2,1,Despite claims from the TV news outlet to offe...,False,0.0
3,1,After receiving 'subpar' service and experienc...,False,0.0
4,1,After watching his beloved Seattle Mariners pr...,False,0.0


In [ ]:
merged_final_df['has_swear_word'].value_counts()

has_swear_word
False    49650
True     10145
Name: count, dtype: int64

In [ ]:
merged_final_df['severity'].value_counts()

severity
0.0     49650
1.0      2123
1.2      1264
1.4       895
1.8       890
        ...  
18.4        1
11.4        1
19.0        1
14.8        1
23.6        1
Name: count, Length: 133, dtype: int64

In [ ]:
# Assuming df is your DataFrame and you want to save it as 'output.csv'
merged_final_df.to_csv('/content/drive/MyDrive/NLP_Project/merged_final_df.csv', index=False)


In [ ]:
merged_final_df = pd.read_csv('/content/drive/MyDrive/NLP_Project/merged_final_df.csv')

In [ ]:
merged_final_df.head()

,label,text,has_swear_word,severity
0,1,"A little less than a decade ago, hockey fans w...",False,0.0
1,1,The writers of the HBO series The Sopranos too...,False,0.0
2,1,Despite claims from the TV news outlet to offe...,False,0.0
3,1,After receiving 'subpar' service and experienc...,False,0.0
4,1,After watching his beloved Seattle Mariners pr...,False,0.0


In [ ]:
#merged_final_df = merged_final_df.drop(columns = ['topic'])

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = merged_final_df['text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
merged_final_df['processed_text'] = preprocessed_corpus


In [ ]:
merged_final_df.head()

,label,text,has_swear_word,severity,processed_text
0,1,"A little less than a decade ago, hockey fans w...",False,0.0,"[little, less, decade, ago, hockey, fans, bles..."
1,1,The writers of the HBO series The Sopranos too...,False,0.0,"[writers, hbo, series, sopranos, took, another..."
2,1,Despite claims from the TV news outlet to offe...,False,0.0,"[despite, claims, tv, news, outlet, offer, non..."
3,1,After receiving 'subpar' service and experienc...,False,0.0,"[receiving, subpar, service, experiencing, unu..."
4,1,After watching his beloved Seattle Mariners pr...,False,0.0,"[watching, beloved, seattle, mariners, prevail..."


In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
#new one
# Create a dictionary representation of the documents
dictionary = Dictionary(merged_final_df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in merged_final_df['processed_text']]

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=5, id2word=dictionary)

# Print the topics and their top words
for idx, topic in lda_model.print_topics(-1, num_words = 20):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.007*"like" + 0.007*"one" + 0.006*"people" + 0.005*"think" + 0.005*"said" + 0.005*"time" + 0.004*"dont" + 0.004*"get" + 0.004*"would" + 0.003*"know" + 0.003*"school" + 0.003*"make" + 0.003*"even" + 0.003*"way" + 0.003*"going" + 0.003*"good" + 0.003*"see" + 0.003*"want" + 0.003*"many" + 0.003*"first"

Topic: 1 
Words: 0.007*"obama" + 0.006*"president" + 0.006*"us" + 0.006*"said" + 0.006*"government" + 0.005*"would" + 0.004*"also" + 0.004*"people" + 0.003*"states" + 0.003*"state" + 0.003*"new" + 0.003*"one" + 0.003*"united" + 0.003*"american" + 0.002*"law" + 0.002*"trump" + 0.002*"public" + 0.002*"military" + 0.002*"country" + 0.002*"political"

Topic: 2 
Words: 0.018*"said" + 0.012*"percent" + 0.011*"taiwan" + 0.008*"year" + 0.006*"million" + 0.005*"billion" + 0.005*"china" + 0.005*"new" + 0.005*"taiwans" + 0.004*"market" + 0.004*"economic" + 0.004*"chinese" + 0.004*"last" + 0.004*"us" + 0.004*"also" + 0.003*"years" + 0.003*"first" + 0.003*"would" + 0.003*"taiwanese" +

Topic 0 - General Opinions and Discussions:

This topic seems to revolve around general discussions and opinions. Words like "like," "think," "said," "people," and "time" suggest a conversation about personal experiences, thoughts, or everyday occurrences. It's likely to cover a wide range of casual topics and could be related to social media discussions or informal conversations.
Topic 1 - Politics and Government Affairs:

This topic is heavily focused on political matters and government affairs. Words like "president," "government," "states," "state," "country," and "political" indicate discussions about political figures, government policies, and state-related issues. It may include discussions about the presidency, legislation, international relations, and political events.
Topic 2 - Economic Discussions:

This topic appears to center around economic discussions. Terms like "percent," "million," "billion," "market," and "economic" suggest conversations related to financial markets, economic indicators, and business news. It may involve discussions about market trends, economic forecasts, financial investments, and business developments.
Topic 3 - Health and Medical Topics:

This topic focuses on health and medical-related discussions. Words like "health," "medical," "research," "water," "food," and "children" indicate discussions related to health research, medical advancements, public health concerns, and possibly environmental health issues. It may cover topics such as healthcare policies, medical breakthroughs, and health education.
Topic 4 - News Events and Incidents:

This topic revolves around news events and incidents. Words like "police," "video," "news," "incident," "court," and "family" suggest discussions related to current events, crime reports, and news coverage. It may involve discussions about various incidents, legal proceedings, breaking news, and media coverage.
Based on this analysis, we can assign a final topic label to each:

Topic 0: General Opinions and Discussions

Topic 1: Politics and Government Affairs

Topic 2: Economic Discussions

Topic 3: Health and Medical Topics

Topic 4: News Events and Incidents

These final topic labels summarize the main themes captured by each topic based on the analysis of the dominant words within each topic.

In [ ]:
import pandas as pd


# Step 2: Calculate percentage of rows per topic within each class
topic_distribution_per_class = merged_final_df.groupby(['label', 'topic']).size().unstack(fill_value=0)
topic_distribution_per_class_percentage = topic_distribution_per_class.div(topic_distribution_per_class.sum(axis=1), axis=0) * 100

# Step 3: Calculate percentage of topics per class
class_distribution_per_topic = merged_final_df.groupby(['topic', 'label']).size().unstack(fill_value=0)
class_distribution_per_topic_percentage = class_distribution_per_topic.div(class_distribution_per_topic.sum(axis=1), axis=0) * 100

print("Percentage of rows per topic within each class:")
print(topic_distribution_per_class_percentage)
print("\nPercentage of topics per class:")
print(class_distribution_per_topic_percentage)


Percentage of rows per topic within each class:
topic          0          1          2          3          4
label                                                       
1      55.977486  13.543745   2.814192   8.741807  18.922770
2      16.894879  48.252746   0.171160   1.447725  33.233490
3      13.530739  49.131416   5.510434  27.490130   4.337281
4      10.587563  34.589848  30.370529   9.016920  15.435140

Percentage of topics per class:
label          1          2          3          4
topic                                            
0      55.691806  16.791891  17.004536  10.511766
1       8.554201  30.445934  39.198128  21.801737
2       6.991150   0.424779  17.292035  75.292035
3      16.215145   2.682701  64.411259  16.690895
4      25.919781  45.476725   7.504636  21.098858


In [ ]:
#new one
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm

# Assuming df is your DataFrame and lda_model is your trained LDA model
# Create a dictionary representation of the documents
dictionary = Dictionary(merged_final_df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in merged_final_df['processed_text']]

# Assign the topic with the highest probability for each document to a new column
topics = []
for doc in tqdm(bow_corpus, desc="Assigning Topics"):
    topics.append(max(lda_model[doc], key=lambda item: item[1])[0])

# Add the 'topic' column to the DataFrame
merged_final_df['topic'] = topics

# Print the DataFrame with the new 'topic' column
merged_final_df.head()


Assigning Topics: 100%|██████████| 59795/59795 [00:50<00:00, 1174.25it/s]


,label,text,has_swear_word,severity,processed_text,topic
0,1,"A little less than a decade ago, hockey fans w...",False,0.0,"[little, less, decade, ago, hockey, fans, bles...",0
1,1,The writers of the HBO series The Sopranos too...,False,0.0,"[writers, hbo, series, sopranos, took, another...",4
2,1,Despite claims from the TV news outlet to offe...,False,0.0,"[despite, claims, tv, news, outlet, offer, non...",4
3,1,After receiving 'subpar' service and experienc...,False,0.0,"[receiving, subpar, service, experiencing, unu...",0
4,1,After watching his beloved Seattle Mariners pr...,False,0.0,"[watching, beloved, seattle, mariners, prevail...",0


In [ ]:
merged_final_df['topic'].value_counts()

topic
1    22223
0    14108
4    10247
3     7567
2     5650
Name: count, dtype: int64

In [ ]:
merged_final_df.to_csv('/content/drive/MyDrive/NLP_Project/merged_final_df_with_topics_new.csv', index=False)


In [ ]:
from gensim.models import LdaModel

# Assuming lda_model is your LDA model
# Save the model
lda_model.save("/content/drive/MyDrive/NLP_Project/lda_model")

# To load the model later, you can use:
# lda_model = LdaModel.load("lda_model")


In [ ]:
merged_final_df = pd.read_csv('/content/drive/MyDrive/NLP_Project/merged_final_df_with_topics_new.csv')

In [ ]:
merged_final_df.head()

,label,text,has_swear_word,severity,processed_text,topic
0,1,"A little less than a decade ago, hockey fans w...",False,0.0,"['little', 'less', 'decade', 'ago', 'hockey', ...",0
1,1,The writers of the HBO series The Sopranos too...,False,0.0,"['writers', 'hbo', 'series', 'sopranos', 'took...",0
2,1,Despite claims from the TV news outlet to offe...,False,0.0,"['despite', 'claims', 'tv', 'news', 'outlet', ...",0
3,1,After receiving 'subpar' service and experienc...,False,0.0,"['receiving', 'subpar', 'service', 'experienci...",4
4,1,After watching his beloved Seattle Mariners pr...,False,0.0,"['watching', 'beloved', 'seattle', 'mariners',...",0


###Applying transformations to text data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/NLP_Project/balancedtest.csv'

column_names = ['label', 'text']

test_df = pd.read_csv(file_path, names=column_names)

test_df.head()


,label,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


In [ ]:
swear_words_df_2 = pd.read_csv('/content/drive/MyDrive/NLP_Project/profanity_en.csv')

In [ ]:
swear_words_dict = dict(zip(swear_words_df_2['canonical_form_1'], swear_words_df_2['severity_rating']))


In [ ]:
import re
from tqdm import tqdm

# Function to calculate total severity rating for a text
def get_total_severity(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive matching
    total_severity = 0
    for word in swear_words_dict:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            total_severity += swear_words_dict[word]
    return total_severity

# Function to check if a text contains any swear words
def contains_swear_words(text):
    text = text.lower()  # Convert text to lowercase for case-insensitive matching
    for word in swear_words_dict:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            return True
    return False

# Add 'has_swear_word' and 'severity' columns to the DataFrame
tqdm.pandas(desc="Applying contains_swear_words")
test_df['has_swear_word'] = test_df['text'].progress_apply(contains_swear_words)
tqdm.pandas(desc="Applying get_total_severity")
test_df['severity'] = test_df['text'].progress_apply(get_total_severity)

# Print the modified DataFrame
test_df.head()

Applying get_total_severity: 100%|██████████| 3000/3000 [01:14<00:00, 40.13it/s]


,label,text,has_swear_word,severity
0,1,When so many actors seem content to churn out ...,False,0.0
1,1,In what football insiders are calling an unex...,True,2.2
2,1,In a freak accident following Game 3 of the N....,False,0.0
3,1,North Koreas official news agency announced to...,False,0.0
4,1,The former Alaska Governor Sarah Palin would b...,False,0.0


In [ ]:
test_df['has_swear_word'].value_counts()

has_swear_word
False    2354
True      646
Name: count, dtype: int64

In [ ]:
test_df['severity'].value_counts()

severity
0.0     2354
1.0      177
1.2       85
1.8       79
2.8       54
1.4       43
2.2       39
1.6       29
3.0       19
2.0       13
4.6       11
3.8        9
4.0        9
2.6        9
2.4        8
3.6        6
3.2        6
4.4        4
4.2        3
6.8        3
2.6        3
6.0        3
9.0        2
6.2        2
6.4        2
4.2        2
5.8        2
5.2        1
23.8       1
8.4        1
10.4       1
12.6       1
7.6        1
13.2       1
15.2       1
5.4        1
5.2        1
14.2       1
8.2        1
18.0       1
6.6        1
13.4       1
51.0       1
7.2        1
3.8        1
12.2       1
7.6        1
7.2        1
4.8        1
3.4        1
5.0        1
Name: count, dtype: int64

Applying LDA to Test Data

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = test_df['text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
test_df['processed_text'] = preprocessed_corpus


In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
#new one
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm

# Assuming df is your DataFrame and lda_model is your trained LDA model
# Create a dictionary representation of the documents
dictionary = Dictionary(test_df['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in test_df['processed_text']]

# Assign the topic with the highest probability for each document to a new column
topics = []
for doc in tqdm(bow_corpus, desc="Assigning Topics"):
    topics.append(max(lda_model[doc], key=lambda item: item[1])[0])

# Add the 'topic' column to the DataFrame
test_df['topic'] = topics

# Print the DataFrame with the new 'topic' column
test_df.head()


Assigning Topics: 100%|██████████| 3000/3000 [00:02<00:00, 1024.04it/s]


,label,text,has_swear_word,severity,processed_text,topic
0,1,When so many actors seem content to churn out ...,False,0.0,"[many, actors, seem, content, churn, performan...",0
1,1,In what football insiders are calling an unex...,True,2.2,"[football, insiders, calling, unexpectedly, se...",4
2,1,In a freak accident following Game 3 of the N....,False,0.0,"[freak, accident, following, game, 3, nba, fin...",4
3,1,North Koreas official news agency announced to...,False,0.0,"[north, koreas, official, news, agency, announ...",4
4,1,The former Alaska Governor Sarah Palin would b...,False,0.0,"[former, alaska, governor, sarah, palin, would...",4


In [ ]:
test_df['topic'].value_counts()

topic
0    2547
4     255
3     124
1      67
2       7
Name: count, dtype: int64

In [ ]:
test_df.to_csv('/content/drive/MyDrive/NLP_Project/test_final_with_topics_new.csv', index=False)

## Applying LDA on results of model predictions

### Neural Networks

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Import libraries
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Load data
data = pd.read_csv("/content/drive/MyDrive/NLP_Project/final_analysis/nn/model_predictions_original_dataset.csv")  # Replace "your_dataframe.csv" with your actual file path


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = data['Text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
data['processed_text'] = preprocessed_corpus


In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from gensim.models import LdaModel

# Load LDA model
lda_model = LdaModel.load("/content/drive/MyDrive/NLP_Project/lda_model")  # Replace "your_lda_model" with your actual file path


In [ ]:
#new one
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm

# Assuming df is your DataFrame and lda_model is your trained LDA model
# Create a dictionary representation of the documents
dictionary = Dictionary(data['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in data['processed_text']]

# Assign the topic with the highest probability for each document to a new column
topics = []
for doc in tqdm(bow_corpus, desc="Assigning Topics"):
    topics.append(max(lda_model[doc], key=lambda item: item[1])[0])

# Add the 'topic' column to the DataFrame
data['topic'] = topics

# Print the DataFrame with the new 'topic' column
data.head()


Assigning Topics: 100%|██████████| 3000/3000 [00:04<00:00, 696.91it/s]


,Text,Original Label,Predicted Label,processed_text,topic
0,When so many actors seem content to churn out ...,1,4,"[many, actors, seem, content, churn, performan...",0
1,In what football insiders are calling an unex...,1,1,"[football, insiders, calling, unexpectedly, se...",4
2,In a freak accident following Game 3 of the N....,1,4,"[freak, accident, following, game, 3, nba, fin...",4
3,North Koreas official news agency announced to...,1,4,"[north, koreas, official, news, agency, announ...",4
4,The former Alaska Governor Sarah Palin would b...,1,4,"[former, alaska, governor, sarah, palin, would...",4


In [ ]:
data['topic'].value_counts()

topic
0    2497
4     331
3     110
1      57
2       5
Name: count, dtype: int64

In [ ]:
# Group by 'Original Label' and 'topic', then count the number of rows in each group
topic_counts_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Print the result
print(topic_counts_per_class)


topic             0   1  2   3    4
Original Label                     
1               581  21  2  29  117
2               631   9  2  15   93
3               659  16  0  32   43
4               626  11  1  34   78


In [ ]:
# Count misclassifications
misclassifications = data[data['Original Label'] != data['Predicted Label']]
misclassification_counts = misclassifications.groupby(['Original Label', 'Predicted Label']).size().unstack(fill_value=0)


In [ ]:
print(misclassification_counts)

Predicted Label   1   2    3    4
Original Label                   
1                 0  54  281  180
2                12   0  678   29
3                 0   0    0    8
4                 7   5  441    0


In [ ]:
# Calculate accuracy of each topic
topic_accuracy = data.groupby(['Original Label', 'topic']).apply(lambda x: (x['Original Label'] == x['Predicted Label']).mean()).unstack(fill_value=0)


In [ ]:
print(topic_accuracy)

#so below is df where each row represents a true label, each column represents a topic, and the value in each cell represents the accuracy of the model in predicting that topic for that true label.

topic                  0         1    2         3         4
Original Label                                             
1               0.313253  0.285714  0.0  0.448276  0.290598
2               0.030111  0.222222  0.0  0.266667  0.064516
3               0.992413  0.937500  0.0  0.968750  0.976744
4               0.349840  0.818182  1.0  0.852941  0.500000


In [ ]:
# Find most misclassified topic

#calculates the topic that the model misclassified the most for each class (Original Label)
most_misclassified_topic = topic_accuracy.idxmin(axis=1)
print(most_misclassified_topic)

Original Label
1    2
2    2
3    2
4    0
dtype: int64


In [ ]:
# Filter rows where the topic is 2
rows_with_topic_2 = data[data['topic'] == 2]

# Print the resulting DataFrame
rows_with_topic_2.head()


,Text,Original Label,Predicted Label,processed_text,topic
215,Shut it down. This kid just won the internet. ...,1,3,"[shut, kid, internet, thirdgrader, brandon, ha...",2
742,Andddd cue the waterworks. For anyone who need...,1,3,"[andddd, cue, waterworks, anyone, needs, cry, ...",2
1009,"Fox: Trump Has Extreme, Sick OBSESSION Yet, ag...",2,3,"[fox, trump, extreme, sick, obsession, yet, do...",2
1285,[Watch] Krauthammer Reams Media For Not Report...,2,3,"[watch, krauthammer, reams, media, reporting, ...",2
2526,Results Thursday from the St. Petersburg Open ...,4,4,"[results, thursday, st, petersburg, open, 6350...",2


these are the full sentences belonging to topic 2

215                                                           
Shut it down. This kid just won the internet. When third-grader Brandon Hamill got to question five on his science test, he might not have known the right answer, but he did have the PERFECT answer. Check out Brandons genius response to a question about the water cycle: Brilliant. Via imgflip.com Brandons teacher may have marked his answer wrong, but if it were up to us, wed give him extra credit for totally making our day. Brandon FTW!

742

Andddd cue the waterworks. For anyone who needs a cry, this oughta get the job done. Youre just going to want to make sure youre not in public when you see himit could get embarrassing. Dont say we didnt warn you. Via Giphy.com Pass the Kleenex, please!

1009    

Fox: Trump Has Extreme, Sick OBSESSION Yet, again, Donald Trump has taken public his unending dislike of Megyn Kelly. He tweeted a suggestion to his followers on Friday. Everybody should boycott the @megynkelly show. Never worth watching. Always a hit on Trump! She is sick, the most overrated person on tv. Officials at Fox did not let the comment go unnoticed. A spokesperson from the network issued a response. Donald Trumps vitriolic attacks against Megyn Kelly and his extreme, sick obsession with her is beneath the dignity of a presidential candidate who wants to occupy the highest office in the land. He then added further reassurance of Foxs stance. Megyn is an exemplary journalist and one of the leading anchors in America were extremely proud of her phenomenal work and continue to fully support her throughout every day of Trumps endless barrage of crude and sexist verbal assaults.As the mother of three young children, with a successful law career and the second highest rated show in cable television, its especially deplorable for her to be repeatedly abused just for doing her job. The spokesman wasnt the only person to chime in on Trumps comment. Bret Baier, Kellys colleague, posted an opinion. Enough. @megynkelly is not crazy-shes not sick. Well continue to cover this race fairly. Even Geraldo Rivera was upset enough to tweet an answer. @realDonaldTrump @megynkelly I love you man but basta. This obsession w @megynkelly is weird unhelpful. Shes doing her job, you do yours. Amen, Mr. Rivera. Trump has been barraging Megyn Kelly with negativity since the first Republican presidential debate. Kelly upset the Donald by asking him about some his past statements about women. Since then, he has continually made his displeasure known. He even went so far as to skip one the debates because she was the moderator. If Trump truly wants to be Commander in Chief, maybe he should try following the military saying: Suck it up. Press on. 2016 Vianna Vaughan source: MSN


1285

[Watch] Krauthammer Reams Media For Not Reporting Truth On ObamaCare Nothing like the master not mincing words. From Dr. Krauthammer: ARVE Error: Element ID could not be build, please report this bug. That is all.


2526

 Results Thursday from the St. Petersburg Open a $63,50AT een o idor ar curs t Peerurks Sors Comlex (seedings in parentheses): Singles Second Round Sergiy Stakhovsky, Ukraine, def. Andrey Kuznetsov, Russia, 6-1, 6-4. Igor Kunitsyn, Russia, def. Michail Elgin, Russia, 6-2, 6-3. Denis Istomin, Uzbekistan, def. Mikhail Youzhny (1), Russia, 6-2, 2-1, retired.
Name: Text, dtype: object

In [ ]:
# Set display options to show full content of the column
pd.set_option('display.max_colwidth', None)

# Print the 'Text' column without truncating
print(rows_with_topic_2['Text'])

215                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
2    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
3    Index([0, 1, 3, 4, 2], dtype='int64', name='to...
4    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
dtype: object


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the total number of rows with each topic in each class label
total_rows_per_topic_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the percentage of misclassified rows for each topic within each class label
percentage_misclassifications_per_topic = (misclassifications_per_topic / total_rows_per_topic_per_class) * 100

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = percentage_misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([2, 1, 4, 0, 3], dtype='int64', name='to...
2    Index([2, 0, 4, 1, 3], dtype='int64', name='to...
3    Index([1, 3, 4, 0, 2], dtype='int64', name='to...
4    Index([0, 4, 1, 3, 2], dtype='int64', name='to...
dtype: object


### Linear SVC

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Import libraries
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Load data
data = pd.read_csv("/content/drive/MyDrive/NLP_Project/final_analysis/linear_svc/linearsvc_model_predictions_original_dataset.csv")  # Replace "your_dataframe.csv" with your actual file path


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = data['Text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
data['processed_text'] = preprocessed_corpus


In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from gensim.models import LdaModel

# Load LDA model
lda_model = LdaModel.load("/content/drive/MyDrive/NLP_Project/lda_model")  # Replace "your_lda_model" with your actual file path


In [ ]:
#new one
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm

# Assuming df is your DataFrame and lda_model is your trained LDA model
# Create a dictionary representation of the documents
dictionary = Dictionary(data['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in data['processed_text']]

# Assign the topic with the highest probability for each document to a new column
topics = []
for doc in tqdm(bow_corpus, desc="Assigning Topics"):
    topics.append(max(lda_model[doc], key=lambda item: item[1])[0])

# Add the 'topic' column to the DataFrame
data['topic'] = topics

# Print the DataFrame with the new 'topic' column
data.head()


Assigning Topics: 100%|██████████| 3000/3000 [00:02<00:00, 1084.44it/s]


,Text,Original Label,Predicted Label,processed_text,topic
0,When so many actors seem content to churn out ...,1,1,"[many, actors, seem, content, churn, performan...",0
1,In what football insiders are calling an unex...,1,1,"[football, insiders, calling, unexpectedly, se...",4
2,In a freak accident following Game 3 of the N....,1,1,"[freak, accident, following, game, 3, nba, fin...",4
3,North Koreas official news agency announced to...,1,4,"[north, koreas, official, news, agency, announ...",4
4,The former Alaska Governor Sarah Palin would b...,1,1,"[former, alaska, governor, sarah, palin, would...",4


In [ ]:
data['topic'].value_counts()

topic
0    2497
4     331
3     110
1      57
2       5
Name: count, dtype: int64

In [ ]:
# Group by 'Original Label' and 'topic', then count the number of rows in each group
topic_counts_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Print the result
print(topic_counts_per_class)


topic             0   1  2   3    4
Original Label                     
1               581  21  2  29  117
2               631   9  2  15   93
3               659  16  0  32   43
4               626  11  1  34   78


In [ ]:
# Count misclassifications
misclassifications = data[data['Original Label'] != data['Predicted Label']]
misclassification_counts = misclassifications.groupby(['Original Label', 'Predicted Label']).size().unstack(fill_value=0)


In [ ]:
print(misclassification_counts)

Predicted Label   1   2    3   4
Original Label                  
1                 0  78   48  43
2                43   0  295  42
3                21  15    0  89
4                22   2   24   0


In [ ]:
# Calculate accuracy of each topic
topic_accuracy = data.groupby(['Original Label', 'topic']).apply(lambda x: (x['Original Label'] == x['Predicted Label']).mean()).unstack(fill_value=0)


In [ ]:
print(topic_accuracy)

#so below is df where each row represents a true label, each column represents a topic, and the value in each cell represents the accuracy of the model in predicting that topic for that true label.

topic                  0         1    2         3         4
Original Label                                             
1               0.772806  0.857143  0.0  0.724138  0.794872
2               0.480190  0.333333  0.5  0.800000  0.548387
3               0.830046  0.937500  0.0  0.906250  0.790698
4               0.939297  0.818182  1.0  1.000000  0.897436


In [ ]:
# Find most misclassified topic

#calculates the topic that the model misclassified the most for each class (Original Label)
most_misclassified_topic = topic_accuracy.idxmin(axis=1)
print(most_misclassified_topic)

Original Label
1    2
2    1
3    2
4    1
dtype: int64


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
2    Index([0, 4, 1, 3, 2], dtype='int64', name='to...
3    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
4    Index([0, 4, 1, 2, 3], dtype='int64', name='to...
dtype: object


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the total number of rows with each topic in each class label
total_rows_per_topic_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the percentage of misclassified rows for each topic within each class label
percentage_misclassifications_per_topic = (misclassifications_per_topic / total_rows_per_topic_per_class) * 100

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = percentage_misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([2, 3, 0, 4, 1], dtype='int64', name='to...
2    Index([1, 0, 2, 4, 3], dtype='int64', name='to...
3    Index([4, 0, 3, 1, 2], dtype='int64', name='to...
4    Index([1, 4, 0, 2, 3], dtype='int64', name='to...
dtype: object


### BERT

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Import libraries
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Load data
data = pd.read_csv("/content/drive/MyDrive/NLP_Project/final_analysis/bert_preds/predictions.csv")  # Replace "your_dataframe.csv" with your actual file path


In [ ]:
# Remove specified substrings from the 'text' column
data['text'] = data['text'].str.replace(r'\[CLS\]|\[SEP\]|\[PAD\]', '', regex=True)


In [ ]:
# Rename columns 'true_label' to 'Original Label' and 'pred_label' to 'Predicted Label'
data = data.rename(columns={'true_label': 'Original Label', 'pred_label': 'Predicted Label'})


In [ ]:
data = data.rename(columns={'text': 'Text'})


In [ ]:
data['Original Label'] += 1
data['Predicted Label'] += 1

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = data['Text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
data['processed_text'] = preprocessed_corpus


In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from gensim.models import LdaModel

# Load LDA model
lda_model = LdaModel.load("/content/drive/MyDrive/NLP_Project/lda_model")  # Replace "your_lda_model" with your actual file path


In [ ]:
#new one
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm

# Assuming df is your DataFrame and lda_model is your trained LDA model
# Create a dictionary representation of the documents
dictionary = Dictionary(data['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in data['processed_text']]

# Assign the topic with the highest probability for each document to a new column
topics = []
for doc in tqdm(bow_corpus, desc="Assigning Topics"):
    topics.append(max(lda_model[doc], key=lambda item: item[1])[0])

# Add the 'topic' column to the DataFrame
data['topic'] = topics

# Print the DataFrame with the new 'topic' column
data.head()


Assigning Topics: 100%|██████████| 2996/2996 [00:01<00:00, 1681.43it/s]


,Unnamed: 0,Text,Original Label,Predicted Label,processed_text,topic
0,0,when so many actors seem content to churn out...,1,2,"[many, actors, seem, content, churn, performan...",0
1,1,in what football insiders are calling an unex...,1,1,"[football, insiders, calling, unexpectedly, se...",4
2,2,in a freak accident following game 3 of the n...,1,1,"[freak, accident, following, game, 3, n, b, fi...",4
3,3,north koreas official news agency announced t...,1,2,"[north, koreas, official, news, agency, announ...",4
4,4,the former alaska governor sarah palin would ...,1,1,"[former, alaska, governor, sarah, palin, would...",4


In [ ]:
data['topic'].value_counts()

topic
0    2599
4     261
3      75
1      46
2      15
Name: count, dtype: int64

In [ ]:
# Group by 'Original Label' and 'topic', then count the number of rows in each group
topic_counts_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Print the result
print(topic_counts_per_class)


topic             0   1   2   3    4
Original Label                      
1               593  20  11  22  104
2               646   8   3  11   79
3               676   9   0  24   41
4               684   9   1  18   37


In [ ]:
# Count misclassifications
misclassifications = data[data['Original Label'] != data['Predicted Label']]
misclassification_counts = misclassifications.groupby(['Original Label', 'Predicted Label']).size().unstack(fill_value=0)


In [ ]:
print(misclassification_counts)

Predicted Label   1    2    3    4
Original Label                    
1                 0  225    3    2
2                13    0  156    6
3                68   14    0  647
4                24   10    1    0


In [ ]:
# Calculate accuracy of each topic
topic_accuracy = data.groupby(['Original Label', 'topic']).apply(lambda x: (x['Original Label'] == x['Predicted Label']).mean()).unstack(fill_value=0)


In [ ]:
print(topic_accuracy)

#so below is df where each row represents a true label, each column represents a topic, and the value in each cell represents the accuracy of the model in predicting that topic for that true label.

topic                  0         1         2         3         4
Original Label                                                  
1               0.703204  0.700000  0.545455  0.681818  0.653846
2               0.761610  0.875000  0.666667  0.727273  0.797468
3               0.028107  0.000000  0.000000  0.000000  0.048780
4               0.954678  0.888889  1.000000  0.944444  0.945946


In [ ]:
# Find most misclassified topic

#calculates the topic that the model misclassified the most for each class (Original Label)
most_misclassified_topic = topic_accuracy.idxmin(axis=1)
print(most_misclassified_topic)

Original Label
1    2
2    2
3    1
4    1
dtype: int64


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
2    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
3    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
4    Index([0, 4, 1, 3, 2], dtype='int64', name='to...
dtype: object


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the total number of rows with each topic in each class label
total_rows_per_topic_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the percentage of misclassified rows for each topic within each class label
percentage_misclassifications_per_topic = (misclassifications_per_topic / total_rows_per_topic_per_class) * 100

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = percentage_misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([2, 4, 3, 1, 0], dtype='int64', name='to...
2    Index([2, 3, 0, 4, 1], dtype='int64', name='to...
3    Index([1, 3, 0, 4, 2], dtype='int64', name='to...
4    Index([1, 3, 4, 0, 2], dtype='int64', name='to...
dtype: object


XGBOOST

### XGBOOST

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Import libraries
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Load data
data = pd.read_csv("/content/drive/MyDrive/NLP_Project/final_analysis/xgboost/XGBoost_original.csv")  # Replace "your_dataframe.csv" with your actual file path


In [ ]:
# Rename columns 'true_label' to 'Original Label' and 'pred_label' to 'Predicted Label'
data = data.rename(columns={'Label': 'Original Label', 'predicted_label': 'Predicted Label'})


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df is your DataFrame containing the 'text' column
#corpus = df['text'].iloc[:100]  # Selecting the first 100 rows
corpus = data['Text']
# Define a function for preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    # text = ' '.join(tokens)
    # but better to lemmatize
    #text = [token.lemma_ for token in nlp(text)] #spacy
    text = tokens
    return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(text) for text in corpus]
data['processed_text'] = preprocessed_corpus


In [ ]:
import spacy
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Load the English model in spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from gensim.models import LdaModel

# Load LDA model
lda_model = LdaModel.load("/content/drive/MyDrive/NLP_Project/lda_model")  # Replace "your_lda_model" with your actual file path


In [ ]:
#new one
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm

# Assuming df is your DataFrame and lda_model is your trained LDA model
# Create a dictionary representation of the documents
dictionary = Dictionary(data['processed_text'])

# Filter out words that appear in less than 2 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Convert the documents into bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in data['processed_text']]

# Assign the topic with the highest probability for each document to a new column
topics = []
for doc in tqdm(bow_corpus, desc="Assigning Topics"):
    topics.append(max(lda_model[doc], key=lambda item: item[1])[0])

# Add the 'topic' column to the DataFrame
data['topic'] = topics

# Print the DataFrame with the new 'topic' column
data.head()


Assigning Topics: 100%|██████████| 3000/3000 [00:02<00:00, 1431.73it/s]


,Original Label,Text,Predicted Label,processed_text,topic
0,1,When so many actors seem content to churn out ...,0,"[many, actors, seem, content, churn, performan...",0
1,1,In what football insiders are calling an unex...,0,"[football, insiders, calling, unexpectedly, se...",4
2,1,In a freak accident following Game 3 of the N....,0,"[freak, accident, following, game, 3, nba, fin...",4
3,1,North Koreas official news agency announced to...,3,"[north, koreas, official, news, agency, announ...",4
4,1,The former Alaska Governor Sarah Palin would b...,0,"[former, alaska, governor, sarah, palin, would...",4


In [ ]:
data['topic'].value_counts()

topic
0    2497
4     331
3     110
1      57
2       5
Name: count, dtype: int64

In [ ]:
# Group by 'Original Label' and 'topic', then count the number of rows in each group
topic_counts_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Print the result
print(topic_counts_per_class)


topic             0   1  2   3    4
Original Label                     
1               581  21  2  29  117
2               631   9  2  15   93
3               659  16  0  32   43
4               626  11  1  34   78


In [ ]:
# Count misclassifications
misclassifications = data[data['Original Label'] != data['Predicted Label']]
misclassification_counts = misclassifications.groupby(['Original Label', 'Predicted Label']).size().unstack(fill_value=0)


In [ ]:
print(misclassification_counts)

Predicted Label    0    1    2    3
Original Label                     
1                518    0   38   48
2                 70  393    0   38
3                 98   11  372    0
4                 36    4   20  690


In [ ]:
# Calculate accuracy of each topic
topic_accuracy = data.groupby(['Original Label', 'topic']).apply(lambda x: (x['Original Label'] == x['Predicted Label']).mean()).unstack(fill_value=0)


In [ ]:
print(topic_accuracy)

#so below is df where each row represents a true label, each column represents a topic, and the value in each cell represents the accuracy of the model in predicting that topic for that true label.

topic                  0         1    2         3         4
Original Label                                             
1               0.209983  0.095238  0.0  0.172414  0.145299
2               0.340729  0.555556  0.5  0.266667  0.258065
3               0.355083  0.375000  0.0  0.375000  0.395349
4               0.000000  0.000000  0.0  0.000000  0.000000


In [ ]:
# Find most misclassified topic

#calculates the topic that the model misclassified the most for each class (Original Label)
most_misclassified_topic = topic_accuracy.idxmin(axis=1)
print(most_misclassified_topic)

Original Label
1    2
2    4
3    2
4    0
dtype: int64


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
2    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
3    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
4    Index([0, 4, 3, 1, 2], dtype='int64', name='to...
dtype: object


In [ ]:
# Group by class label and topic, then count the number of misclassifications in each group
misclassifications_per_topic = data[data['Original Label'] != data['Predicted Label']].groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the total number of rows with each topic in each class label
total_rows_per_topic_per_class = data.groupby(['Original Label', 'topic']).size().unstack(fill_value=0)

# Calculate the percentage of misclassified rows for each topic within each class label
percentage_misclassifications_per_topic = (misclassifications_per_topic / total_rows_per_topic_per_class) * 100

# Rank the topics within each class label from most misclassified to least misclassified
ranked_topics_per_class = percentage_misclassifications_per_topic.apply(lambda x: x.sort_values(ascending=False).index, axis=1)

# Print the result
print(ranked_topics_per_class)


Original Label
1    Index([2, 1, 4, 3, 0], dtype='int64', name='to...
2    Index([4, 3, 0, 2, 1], dtype='int64', name='to...
3    Index([0, 1, 3, 4, 2], dtype='int64', name='to...
4    Index([0, 1, 2, 3, 4], dtype='int64', name='to...
dtype: object


METEOR SCORE FOR PARAPHRASING EVALUATION

In [ ]:
df_aug_2 = pd.read_csv('/content/drive/MyDrive/NLP_Project/2_train_aug.csv')


In [ ]:
df_aug_2.describe()

,Label
count,4682.000000
mean,3.000000
std,1.000107
min,2.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,4.000000


In [ ]:
df_aug_2.head()

,Label,Text
0,2,Montel Williams Voices His Opinion On What Sho...
1,2,"Marines Welcome George W. Bush, But Are Silent..."
2,2,BREAKING: Hillary Scrambling After Fox News Bl...
3,2,Muslims Fight To Ban Christianity From The Mil...
4,2,Hillary Clinton Caught Doing THIS With The Sau...


In [ ]:
df_aug_2['Label'].value_counts()

Label
2    2341
4    2341
Name: count, dtype: int64

In [ ]:
length = len(df_aug_2)

# Calculate the midpoint
midpoint = length // 2

# Split the DataFrame into two halves
first_half = df_aug_2.iloc[:midpoint]
second_half = df_aug_2.iloc[midpoint:]

In [ ]:
first_half['Label'].value_counts()

Label
4    1184
2    1157
Name: count, dtype: int64

In [ ]:
second_half['Label'].value_counts()

Label
2    1184
4    1157
Name: count, dtype: int64

In [ ]:
print(first_half.head())

   Label                                               Text
0      2  Montel Williams Voices His Opinion On What Sho...
1      2  Marines Welcome George W. Bush, But Are Silent...
2      2  BREAKING: Hillary Scrambling After Fox News Bl...
3      2  Muslims Fight To Ban Christianity From The Mil...
4      2  Hillary Clinton Caught Doing THIS With The Sau...


In [ ]:
print(second_half.head())

      Label                                               Text
2341      4  As the Senate considers historic reforms to he...
2342      4  The corner of two local streets in Compton mig...
2343      4  I've just gotten on the elevator, and I have a...
2344      4  Disney Pixar Animation guru John Lasseter foun...
2345      4  That's according to a population-based study p...


In [ ]:
# Split the DataFrame into two separate DataFrames based on label values
df_label_2 = df_aug_2[df_aug_2['Label'] == 2]
df_label_4 = df_aug_2[df_aug_2['Label'] == 4]


In [ ]:
df_label_2.head()

,Label,Text
0,2,Montel Williams Voices His Opinion On What Sho...
1,2,"Marines Welcome George W. Bush, But Are Silent..."
2,2,BREAKING: Hillary Scrambling After Fox News Bl...
3,2,Muslims Fight To Ban Christianity From The Mil...
4,2,Hillary Clinton Caught Doing THIS With The Sau...


In [ ]:
df_label_4.head()

,Label,Text
1157,4,UN chief Ban Ki-moon on Friday congratulated U...
1158,4,US President Barack Obama's winning of the Nob...
1159,4,US envoy George Mitchell was to meet Israeli P...
1160,4,South Korea and Japan called Friday for a 'fun...
1161,4,Three people arrested after Iran's disputed pr...


In [ ]:
import numpy as np

# Split df_label_2 into two separate DataFrames
df_label_2_part1, df_label_2_part2 = np.array_split(df_label_2, 2)


In [ ]:
df_label_2_part1.head()

,Label,Text
0,2,Montel Williams Voices His Opinion On What Sho...
1,2,"Marines Welcome George W. Bush, But Are Silent..."
2,2,BREAKING: Hillary Scrambling After Fox News Bl...
3,2,Muslims Fight To Ban Christianity From The Mil...
4,2,Hillary Clinton Caught Doing THIS With The Sau...


In [ ]:
df_label_2_part2.head()

,Label,Text
2809,2,During an interview for Super Bowl XLIX Media...
2810,2,"According to recent information, the US milit..."
2811,2,"Responding to recent events, Israeli Prime Mi..."
2812,2,"Open Carry Texas supporters, advocating for t..."
2813,2,Officer Wilson's attorney has shared that the...


In [ ]:
df_label_2_part1.describe()

,Label
count,1171.0
mean,2.0
std,0.0
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0


In [ ]:
df_label_2_part2.describe()

,Label
count,1170.0
mean,2.0
std,0.0
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0
